In [1]:
#Для расчета трассы полета
!pip install pyorbital
#Для получения TLE файла из базы
!pip install spacetrack

#Для работы с шейп-файлами
!pip install pyshp
!pip install shapely
!pip install shapefile

#Для работы с геоданными
!pip install geopandas
!pip install contextily
!pip install fiona

#Для создания интерактивной карты
!pip install folium

ERROR: Could not find a version that satisfies the requirement shapefile (from versions: none)
ERROR: No matching distribution found for shapefile


In [5]:
#Библиотеки для построения трассировки КА
from datetime import datetime, date, timedelta
import spacetrack.operators as op
from spacetrack import SpaceTrackClient
from pyorbital.orbital import Orbital

#Библиотеки для работы с шейпфайлами (shapefiles)
import shapefile
import fiona
from shapely.geometry import Point, Polygon, mapping

#Библиотеки для математических вычислений и моделирования
import numpy as np
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import contextily as ctx
import pandas as pd
import csv
import math

#Библиотека folium для создания интерактивной карты
import folium
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon
print('All installed and imported!')

All installed and imported!


In [7]:
def energy_consumption(P_ust, P_obes, N_radio, T_KA, t_sem, input_csv):
    '''file=gpd.read_file(input_csv)
    df=pd.DataFrame(file)'''
    df = pd.read_csv(input_csv)
    nv=[]
    str=[]
    time_sec=0
    P_sem=0
    P_mv=0
    P_no=0
    P_o=0
    for i in range(len(df)):
        str=df['TIME'][i]
        time_sec=int(str.split()[1].split(sep=':')[0])*3600+int(str.split()[1].split(sep=':')[1])*60+int(str.split()[1].split(sep=':')[2])
        nv.append(1+(time_sec//T_KA))
    actuals_s = pd.Series(nv)
    df['VITOK'] = actuals_s
    j=0
    for i in range(1, df['VITOK'].max()+1):
        k=0
        P_sem=0
        while j<len(df) and i==df['VITOK'][j]:
            j+=1
            k+=1
        P_sem=k*N_radio*t_sem/T_KA
        P_sem=round(P_sem, 3)
        P_mv=0
        P_no=0
        P_o=0
        n=int(input('Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций'))
        if n!=0:
            for g in range(n):
                k=int(input('Введите количество систем участвующих в {}-й операции'.format(g+1)))
                for f in range(k):
                    N_k=float(input('Введите энергопотребление {}-й системы'.format(f+1)))
                    t_k=int(input('Введите время работы {}-й системы'.format(f+1)))
                    P_no+=N_k*t_k
        n=int(input('Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций'))
        if n!=0:
            for g in range(n):
                k=int(input('Введите количество систем участвующих в {}-й операции'.format(g+1)))
                for f in range(k):
                    N_k=float(input('Введите энергопотребление {}-й системы'.format(f+1)))
                    t_k=int(input('Введите время работы {}-й системы'.format(f+1)))
                    P_o+=N_k*t_k
        P_mv=P_obes+P_no+P_o+P_sem
        print("Энергопотребление на витке:", P_mv)
        if P_mv<=P_ust:
            print('{} - OK'.format(i))
        else:
            print('{} - энергопотребление превышено'.format(i))

In [11]:
papka = str(input('Введите путь к рабочей папке:'))
P_ust = float(input('Введите установленную средневитковую мощность:'))
P_ob = float(input('Введите мощность обеспечивающих систем в дежурном режиме:'))
N =  float(input('Введите мощность радиолокатора:'))
t = int(input('Введите время продолжительности съёмки (в сек):'))
T = int(input('Период обращения КА (в сек):'))

Введите путь к рабочей папке: C:/Users/Sirius/Desktop/Тест единства
Введите установленную средневитковую мощность: 310.5
Введите мощность обеспечивающих систем в дежурном режиме: 230.6
Введите мощность радиолокатора: 73.8
Введите время продолжительности съёмки (в сек): 15
Период обращения КА (в сек): 5400


In [12]:
energy_consumption(P_ust, P_ob, N, T, t, papka + '/shooting_time_all.csv')

Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций 0
Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций 0


Энергопотребление на витке: 231.01
1 - OK


Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций 0
Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций 0


Энергопотребление на витке: 230.805
2 - OK


Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций 0
Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций 0


...

Энергопотребление на витке: 231.42
14 - OK


Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций 0
Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций 0


Энергопотребление на витке: 231.215
15 - OK


Начинаем расчёт энергопотребления не отменяемых операций. Введите количество операций 0
Начинаем расчёт энергопотребления отменяемых операций. Введите количество операций 0


Энергопотребление на витке: 231.215
16 - OK
